<a href="https://colab.research.google.com/github/K-Jadeja/HuggingFace-LLMs/blob/main/(Compromise)WizardLM_7B_No_OpenAI_HF_Instructor_Embeddings_Chroma_DB_Multi_Doc_Retriever_LangChain_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Result**: 5/5
## Works exceptionally well. But needs high RAM
## Compromise with instructor-large : SysRAM- 5.8G GPURAM- 11.1G Disk- 43.9G


In [ ]:
!pip install torch==2.0.1
!pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
!pip -q install accelerate bitsandbytes sentencepiece Xformers

In [ ]:
!pip show langchain

## QA Retrieval No Open AI - WizardLM




Importing research articles form TechCrunch...

In [2]:
!wget -q https://www.dropbox.com/s/zoj9rnm7oyeaivb/new_papers.zip
!unzip -q new_papers.zip -d new_papers

In [4]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/wizardLM-7B-HF")

model = LlamaForCausalLM.from_pretrained("TheBloke/wizardLM-7B-HF",
                                              load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True
                                              )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [6]:
print(local_llm('What is the capital of England?'))


London is the capital city of England.


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files - PDFs
- ChromaDB
- Local LLM
- Instuctor Embeddings


## Setting up LangChain 


In [7]:
import os



In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## Load multiple and process documents

In [ ]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')

In [ ]:
# Loading and processing single text file

# loader = TextLoader('single_text_file.txt')

* glob parameter is used to control which files to load. Note that here it doesn’t load the .rst file or the .ipynb files.
* use loader_cls=TextLoader if we are loading text files instead of pdf




In [6]:
# Loading and processing multiple text files

loader = DirectoryLoader('./new_papers/new_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()
pages = loader.load_and_split()

In [10]:
len(pages)

185

In [5]:
len(documents)

142

In [11]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## HF Instructor Embeddings
(for some reason, instructor-xl uses ststem ram which leads to crash. Using instructor-large instead which led to GPU usage to shoot to 11.1 GB)


In [12]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initializing embeddings
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large", 
                                                      model_kwargs={"device": "cuda"})
# "device":"CPU"  if you want to run it on cpu


load INSTRUCTOR_Transformer
max_seq_length  512


## create the DB
**Using CromaDB for vector store**

In [13]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the new embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Making a retriever

search_kwargs={"k":3} means retriever retrieves top 3 most similar texts(splitted documents) so we can feed them into our LLM as context. This way, we don't exceed our token limit

In [14]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

## Creating a chain

In [15]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=local_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)
# we can set verbose = True if we want to see whats happening in the background

In [16]:
## printing the result that we get from the query nicely and citing sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [17]:
# full example
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 FlashAttention is a new type of attention mechanism designed to be fast and memory-efficient while still
maintaining high accuracy. It was introduced in a research paper published on June 24th, 2022, by Tri Daoy,
Daniel Y. Fuy, Stefano Ermon, Atri


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [18]:
# break it down
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

 IO-aware refers to a methodology that takes into account the input/output (IO) operations of a system when
designing hardware or software solutions. In the context of deep learning, it means considering the impact of
IO operations on the performance and efficiency of neural networks.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [19]:
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Tiling in FlashAttention refers to the process of dividing the attention matrix into smaller sub-matrices or
"tiles" instead of materializing the entire matrix in memory. This reduces the memory footprint and allows for
faster computation as fewer memory reads are required.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [20]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer is a language model that has been trained to use external tools such as search engines,
calculators, and translation systems via simple API calls. It was introduced in a research paper published
recently by Timo Schick, Jane Dwivedi-Yu, Roberto Dessìy, Roberta Raileanu, Maria Lomeli, Luke Zettlemoyer,
Nicola Cancedda, Thomas Scialom, Meta AI Research, and Universitat Pompeu Fabra.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [21]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Toolformer can learn to use external tools via simple APIs, so it depends on the APIs that are available and
accessible. The model learns which tools to use and when to use them based on demonstrations, so it cannot use
tools that are not available or accessible.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [22]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 It depends on the complexity of the task and the level of expertise of the user. Typically, more examples are
needed for less common or complex tools. However, some users may be able to learn and use new tools quickly
with fewer examples.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf


In [23]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

There are several different types of retrieval augmentations that can be used for LLMs, including dense and
sparse retrievers. Dense retrievers work with dense bag-of-words representations of documents and queries,
while sparse retrievers use sparse representations of documents and queries. Both approaches can be used to
augment LLMs, and the choice depends on the specific task and dataset. Additionally, some researchers have
proposed using external tools such as calculators or interpreters to improve the accuracy of LLMs. Overall,
the goal of retrieval-augmented LLMs is to reduce the amount of parameter required to achieve comparable
performance to larger LLMs while still maintaining the ability to reason about complex questions.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [24]:
query = "What are the differences between REALM and RAG?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

According to Table 2 from Borgeaud et al. (2022), REALM uses pre-trained frozen BERT embeddings for the
retriever, while RAG fine-tunes the retriever with a sequence-to-sequence model. Additionally, REALM processes
many retrieved documents using an auto-regressive LM, whereas RAG uses chunked cross-attention.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [28]:
query = "What are limitations of reinforcement learning?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Reinforcement learning has several limitations, including:
1. Limited data availability: In many real-world scenarios, there may not be enough data available to train a
reinforcement learning agent effectively. This can lead to suboptimal results or even failure to converge to a
good solution.
2. Computational complexity: Reinforcement learning algorithms can be computationally expensive, particularly
when dealing with high-dimensional state spaces or long time horizons. This can limit their applicability to
certain domains or hardware configurations.
3. Robustness to local optima: Reinforcement learning agents can get stuck in local optima, where they may
achieve good performance on a particular task but fail to generalize to other related tasks. This can be
mitigated through techniques such as exploration and diversity-promoting mechanisms, but they can still be a
challenge in practice.
4. Interpretability: Reinforcement learning models can be difficult to interpret, making it chall

In [25]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7fb1ee3f8640>)

In [26]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
